in the end, we want for each hour the following:
- DE day-ahead price (+ market closing time)
- CH day-ahead price (+ market closing time)
- JAO Auction price DE->CH (+ market closing time)
- JAO Auction price (CH->DE) (+ market closing time)
- for each market closing time some additional predicted weather data or other potential explaining variables

0. make sure, you have the some day-ahead price data for both countries in data/raw
1. with functions from src/upstream.py we get the data from JAO and put it in data/external
2. create a dataset with day-ahead price difference CH-DE per hour, both auction prices, and further variables for that hour that were known at market closing time.

In [1]:
import sys; sys.path.append("..")

import pandas as pd
import requests
import ast
from datetime import datetime, timedelta
from calendar import monthrange

In [2]:
# Import the fetch_auction_data function
from src.datafeed.upstream import fetch_auction_data

from src.config import KEY

Download the data from JAO (This data is in CET/CEST!)

The dataframe in the first year start on 2th January.

In [3]:
# Define your API key (replace with your actual API key)
api_key = KEY

# Set the start and end years
start_year = 2016
end_year = 2022

corridors = ["de-ch", "ch-de"]

for corridor in corridors:

    # Call the function to fetch and process the data
    auction_data = fetch_auction_data(start_year, end_year, api_key, corridor)

    # Optionally, display the first few rows of the combined DataFrame
    auction_data.head()


/Users/research/Desktop/electricity-price-disparity-ch-de/notebooks/../src/datafeed/upstream.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)
/Users/research/Desktop/electricity-price-disparity-ch-de/notebooks/../src/datafeed/upstream.py:75: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(dfs, ignore_index=True)


Processing the data.
x until now, all data is in cet/cest. 

To do:
- DE-LU is in quarter-hours. we should weigh the prices with the forecasted loads so we also get hourly data there. 
- make functions in src/datafeed/downstream.py to create a dataframe in data/processed
    - add data for forecasted loads for both regions (this is before the market closes, estimate by the Transmission System Operators)
    - add hourly forecasted weather data (?)
    - add other forecasted data (?)
    - put all in one easily accessible dataframe for the analysis.

In [ ]:
# Collect day-ahead prices into a single DataFrame (WIP)

years = range(2015, 2023)  # Adjust years as necessary
path = "../data/raw/"  # Update with your actual path

bidding_zones = ["CH", "DE-LU", "DE-AT-LU"]
bidding_zones_others = ["DE-LU", "DE-AT-LU"]
years = [2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

# Dictionary to store dataframes
dataframes = {}

# Dictionary to store concatenated dataframes for each bidding zone
df = {}

for bidding_zone in bidding_zones:
    # List to hold dataframes for a particular bidding zone
    dfs = []
    for year in years:
        # Create a unique key for each dataframe
        try:
            key = f"{bidding_zone}_{year}"
            file_name = f"{path}{bidding_zone}_Day-ahead Prices_{year}.csv"

            # import csv file and store it in the dictionary
            dataframes[key] = pd.read_csv(file_name)
            dfs.append(dataframes[key])
        except:
            pass
    # concatenating dataframes for each bidding zone
    df[bidding_zone] = pd.concat(dfs)

    # Ensure the index is a datetime
    df[bidding_zone]["MTU (CET/CEST)"] = pd.to_datetime(
        df[bidding_zone]["MTU (CET/CEST)"].str.split(" - ").str[0],
        dayfirst=True,
        errors="coerce",
    )
    df[bidding_zone].sort_values(by="MTU (CET/CEST)", inplace=True)
    df[bidding_zone].set_index("MTU (CET/CEST)", inplace=True)
    df[bidding_zone]["Day-ahead Price [EUR/MWh]"] = pd.to_numeric(
        df[bidding_zone]["Day-ahead Price [EUR/MWh]"], errors="coerce"
    )
